In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import stanza
import pandas as pd
import numpy as np

import codecs
import numpy as np
import nltk
import pycrfsuite
from bs4 import BeautifulSoup as bs
from bs4.element import Tag
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
import re
import random
from tqdm.notebook import tqdm, trange

from stanza.utils.conll import CoNLL
from tqdm.notebook import tqdm
import pickle

In [3]:
with open('sentences_tok.pkl', 'rb') as f:
    sentence_tok_list = pickle.load(f)

In [4]:
print(len(sentence_tok_list))

10974


In [5]:
rules_count = dict()
iob_list = list()

def checkRule(tok_list, parent):
#     print(tok_list)
    rule_1_words = list()
    rule_2_words = list()
    rule_3_words = list()
    rule_4_words = list()
    rule_5_words = list()
    
    children = parent['children']
    
    # You comment/uncomment any rule here and it won't be executed
    rules = [
#         'rule_1', 
        'rule_2', 
#         'rule_3',
        'rule_4',
        'rule_5'
    ]
    
    
    # ==========================================================================================================================================================================================
    # Rule 1
    # ==========================================================================================================================================================================================                    
    if ('rule_1' in rules) and set((rule_1 := {id: tok_list[id]['deprel'] for id in children if tok_list[id]['deprel'] in ['compound', 'conj']}).values()) == set(['compound', 'conj']):
        if ((len(rule_1.values()) > 0) and (all(x in [tok_list[id]['deprel'] for id in children] for x in rule_1.values()))):
            rule_1_words = []
            rule_1_words.append(str(tok_list[list(rule_1.keys())[list(rule_1.values()).index('compound')]]['text']) + " " + str(parent['text']))
            for id in rule_1.keys():
                if (tok_list[id]['deprel'] in ['conj']):
                    rule_1_words.append(tok_list[id]['text'] + " " + parent['text'])    
    
    # ==========================================================================================================================================================================================
    # Rule 2
    # ==========================================================================================================================================================================================                    
    if ('rule_2' in rules) and set((rule_2 := {id: tok_list[id]['deprel'] for id in children if tok_list[id]['deprel'] in ['compound', 'amod']}).values()) == set(['compound', 'amod']):
        if ((len(rule_2.values()) > 0) and (all(x in [tok_list[id]['deprel'] for id in children] for x in rule_2.values()))):
            rule_2_words = []
            word_string = ""
            

            for id in range(min(rule_2.keys()), max(rule_2.keys()) + 1):
                if (tok_list[id]['text'] in ['=', ',', '"', '\'']):
                    continue
                word_string+=tok_list[id]['text'] + " "
            word_string+=parent['text']
            rule_2_words.append(word_string)
    
    # ==========================================================================================================================================================================================
    # Rule 3
    # ==========================================================================================================================================================================================                    
    if ('rule_3' in rules) and set((rule_3 := {id: tok_list[id]['deprel'] for id in children if tok_list[id]['deprel'] in ['amod']}).values()) == set(['amod']) and (parent['deprel'] == 'conj'):
        if ((len(rule_3.values()) > 1) and (all(x in [tok_list[id]['deprel'] for id in children] for x in rule_3.values()))):
            rule_3_words = []
            word_string = ""
            for id in rule_3.keys():
                if (tok_list[id]['deprel']) in ['amod']:
                    if (tok_list[id]['text'] in ['=']):
                        continue
                    word_string+=tok_list[id]['text'] + " "
            word_string+=parent['text']
            
            for id in children:
                if tok_list[id]['deprel'] == 'acl':
                    new_tok = tok_list[id]
                    word_string+= " " + new_tok['text']
                    if len(new_tok['children']) > 0:
                        newer_tok = tok_list[new_tok['children'][0]]
                        for i in newer_tok['children']:
                            newest_tok = tok_list[i]
                            if newest_tok['deprel'] == 'case':
                                word_string+=" " + newest_tok['text'] + " " + newer_tok['text']
            
            rule_3_words.append(word_string)

            
    # ==========================================================================================================================================================================================
    # Rule 4
    # ==========================================================================================================================================================================================                    
    if ('rule_4' in rules) and set((rule_4 := {id: tok_list[id]['deprel'] for id in children if tok_list[id]['deprel'] in ['compound']}).values()) == set(['compound']):
        if ((len(rule_4.values()) > 0) and (all(x in [tok_list[id]['deprel'] for id in children] for x in rule_4.values()))):
            rule_4_words = []
            word_string = ""            
            for id in range(min(rule_4.keys()), max(rule_4.keys()) + 1):
                if (tok_list[id]['text'] in ['=', ',', '"', '\'']):
                    continue
                word_string+=tok_list[id]['text'] + " "
            word_string+=parent['text']
            rule_4_words.append(word_string)
            

    # ==========================================================================================================================================================================================
    # Rule 5
    # ==========================================================================================================================================================================================    
    if ('rule_5' in rules) and set((rule_5 := {id: tok_list[id]['deprel'] for id in children if tok_list[id]['deprel'] in ['nmod:poss']}).values()) == set(['nmod:poss']):
        if ((len(rule_5.values()) > 0) and (all(x in [tok_list[id]['deprel'] for id in children] for x in rule_5.values()))):
            if (len(rule_5.values()) == 1):
                current_id = list(rule_5.keys())[0]
                if (tok_list[current_id+1]['deprel'] == 'case') and (tok_list[current_id]['text'].istitle()) and (len(tok_list[current_id+1]['text'])>1):
                    for key in tok_list.keys():
                        print(tok_list[key])
                    word_string = ""
                    word_string+=tok_list[current_id]['text'] + " " + tok_list[current_id+1]['text']
                    for i in range(current_id+2, parent['id']+1):
                        if (tok_list[i]['text'] in ['=', ',', '"', '\'']):
                            continue
                        word_string+=" " + tok_list[i]['text']
                    rule_5_words.append(word_string)
                
    # ==========================================================================================================================================================================================
    # Finding best rule and counting the distribution of triggered rules
    # ==========================================================================================================================================================================================
    words = [
        rule_1_words,
        rule_2_words,
        rule_3_words,
        rule_4_words,
        rule_5_words
    ]
    
    word_dict = {id+1: words[id][0] for id in range(len(words)) if len(words[id]) > 0 and (words[id][0] != [])}
    
    sentence = ' '.join([tok_list[key]['text'] for key in tok_list.keys()])
        
    if (len(word_dict.values()) > 0):
        
        print(sentence)
        print(word_dict)
        
        value = max(word_dict.values(), key=len)
        key = list(word_dict.keys())[list(word_dict.values()).index(value)]
        
        rule_name = 'rule_' + str(key)
        
        if rule_name in rules_count:
            rules_count[rule_name] = rules_count[rule_name] + 1
        else:
            rules_count[rule_name] = 1
        
        print('Rule ' + str(key) + ": ", end='')
        print(value)
        
        print('\n\n')
        
        return value, sentence
    return None, sentence

            
def convertWord2WordDict(sentence):
    return {word['id']:word for word in sentence}

def getWordsFromDepTree(tok_list, parent = None):
#     dependency_list = list(['acl'])
    initial_word_list = list([
        'method', 
        'analysis', 
        'algorithm', 
        'approach', 
        'model', 
        'preparation', 
        'strategy', 
        'technique', 
        'normalization', 
        'quantification',
        'test', 
        'scoring', 
        'score', 
        'statistic'
    ])
#     initial_word_list = list(['statistic'])
     
    if parent == None:
        for tok in tok_list.values():
            if tok['deprel'] == 'root':
                parent = tok
                getWordsFromDepTree(tok_list, parent)
    else:
        for i in parent['children']:
            tok = tok_list[i]
    # ==========================================================================================================================================================================================
    # Uncomment if you don't want limits on suffix words
#             checkRule(tok_list, tok)
#             getWordsFromDepTree(tok_list, tok)
    # ==========================================================================================================================================================================================
    # Uncomment if you want limits on suffix words
            if tok['text'].lower() in initial_word_list:
                value, sentence = checkRule(tok_list, tok)
                
                sentence_list = sentence.split(" ")
                
                inner_iob_list = [0] * len(sentence_list)
                
                xpos_list = [tok['xpos'] for tok in list(tok_list.values())]
                    
                if (value != None):
                    word_list = value.split(" ")
                    
                    for i in range(len(sentence_list)):
                        if (sentence_list[i] == word_list[0]):
                            number_of_matched = 0
                            for j in range(len(word_list)):
                                if (i+j < len(sentence_list)):
                                    if (sentence_list[i+j] == word_list[j]):
                                        number_of_matched+=1
                            if (number_of_matched == len(word_list)):
                                for j in range(len(word_list)):
                                    inner_iob_list[i+j] = j+1
#                     print(inner_iob_list)
                    
                    for index in range(0, len(inner_iob_list)):
                        if inner_iob_list[index] == 0:
                            inner_iob_list[index] = 'O'
                        elif inner_iob_list[index] == 1:
                            inner_iob_list[index] = 'B-method'
                        else:
                            inner_iob_list[index] = 'I-method'

                    list_with_word_and_tag = list(zip(sentence_list, xpos_list, inner_iob_list))
#                     print(list_with_word_and_tag)
                    iob_list.append(list_with_word_and_tag)
            else:
                getWordsFromDepTree(tok_list, tok)
                
# ==============================================================================================================================================================================================

for sentence in sentence_tok_list:
    getWordsFromDepTree(convertWord2WordDict(sentence))
print(rules_count)
print("Total number of sentences: " + str(len(sentence_tok_list)))    


Frozen sections of various green fluorescent protein transgenic mouse heads were prepared using the film method developed by Kawamoto and Shimizu .
{4: 'film method'}
Rule 4: film method



For example , the film method is useful for chasing donor cells that are labeled with GFP in an acceptor body .
{4: 'film method'}
Rule 4: film method



We have previously reported on a rat lymph node method .
{4: 'rat lymph node method'}
Rule 4: rat lymph node method



Sequence analysis of the 16S rRNA gene has proved extremely useful for evolutionary studies of prokaryotes .
{4: 'Sequence analysis'}
Rule 4: Sequence analysis



The effect of different treatments on cell-mediated immune response was examined by the cutaneous basophilic hypersensitivity test .
{2: 'cutaneous basophilic hypersensitivity test', 4: 'hypersensitivity test'}
Rule 2: cutaneous basophilic hypersensitivity test



The Principal Component Analysis technique was used to summarize and discover patterns of inter-correlations 

{'id': 10, 'text': ',', 'lemma': ',', 'upos': 'PUNCT', 'xpos': ',', 'head': 9, 'deprel': 'punct', 'misc': 'start_char=57|end_char=58', 'children': []}
{'id': 11, 'text': 'log2', 'lemma': 'log2', 'upos': 'ADJ', 'xpos': 'JJ', 'head': 13, 'deprel': 'amod', 'misc': 'start_char=59|end_char=63', 'children': []}
{'id': 12, 'text': 'transformed', 'lemma': 'transform', 'upos': 'VERB', 'xpos': 'VBN', 'head': 13, 'deprel': 'amod', 'misc': 'start_char=64|end_char=75', 'children': []}
{'id': 13, 'text': 'data', 'lemma': 'datum', 'upos': 'NOUN', 'xpos': 'NNS', 'head': 9, 'deprel': 'obj', 'misc': 'start_char=76|end_char=80', 'children': [11, 12, 15]}
{'id': 14, 'text': 'from', 'lemma': 'from', 'upos': 'ADP', 'xpos': 'IN', 'head': 15, 'deprel': 'mark', 'misc': 'start_char=81|end_char=85', 'children': []}
{'id': 15, 'text': 'replicate', 'lemma': 'replicate', 'upos': 'VERB', 'xpos': 'VB', 'head': 13, 'deprel': 'acl', 'misc': 'start_char=86|end_char=95', 'children': [14, 16, 20]}
{'id': 16, 'text': 'bead

The four electrode technique used in this study is considered the standard in biophysical sciences and is widely employed to measure electrical conductance / impedance of biological tissue .
{4: 'electrode technique'}
Rule 4: electrode technique



In consultation with the executive members of these associations , all 4,045 names of the members of the associations were obtained through stratified sampling and a probability sampling method was used to select 600 respondents for the questionnaire survey .
{4: 'probability sampling method'}
Rule 4: probability sampling method



Treatment GroupControl GroupTreatment Effect SizeControl Effect SizeComparison Effect SizeR2Effect Size f2Sample SizeVickersBadger1.020.001.020.210.2740VickersSMLGC-W.AVG0.800.160.640.090.10101Vickers90 % C-W.AVG0.620.160.460.050.05187A-W.AVGC-W.AVG0.560.160.400.040.04235Vickers90 % Carpenter0.620.370.250.020.02476A-W.AVGCarpenter0.560.370.190.010.01957A final method we employ is to compare the two weighted averag

In this work , we have developed a theoretical approach to estimate allele frequency in pooled DNA samples , based on the physical principles of DNA immobilization and hybridization on solid surface using the Langmuir kinetic model and quantitative analysis of the allelic signals .
{2: 'Langmuir kinetic model', 4: 'Langmuir model'}
Rule 2: Langmuir kinetic model



The MCL algorithm was applied to find clusters in this graph .
{4: 'MCL algorithm'}
Rule 4: MCL algorithm



This study examined the patterns of change in ADL for those residents with moderate to severe cognitive impairment therefore the CPS was used to identify residents in two groups : those with a CPS score of 3 and classified as moderate impairment and those with a CPS score of 4 or 5 and classified as severe impairment .
{4: 'CPS score'}
Rule 4: CPS score



This study examined the patterns of change in ADL for those residents with moderate to severe cognitive impairment therefore the CPS was used to identify residents 

Recently , we described a new high throughput AFLP approach for the identification of DNA polymorphism in Mycobacterium tuberculosis .
{2: 'new high throughput AFLP approach', 4: 'throughput AFLP approach'}
Rule 2: new high throughput AFLP approach



Out of 41 naturally contaminated infant formulae and environmental samples analysed for the presence of E. sakazakii , 23 were positive by real-time PCR and 22 by the conventional culture method , giving 97.5 % concordance with the ISO-IDF reference method .
{2: 'conventional culture method', 4: 'culture method'}
Rule 2: conventional culture method



Out of 41 naturally contaminated infant formulae and environmental samples analysed for the presence of E. sakazakii , 23 were positive by real-time PCR and 22 by the conventional culture method , giving 97.5 % concordance with the ISO-IDF reference method .
{4: 'ISO-IDF reference method'}
Rule 4: ISO-IDF reference method



The ARDRA method demonstrated to be useful for intraspecific analys

...


The genome-wide admixture mapping analysis was only performed in African-Americans since the SNP set of Smith et al. was specifically selected for the African-American population .
{2: 'genome-wide admixture mapping analysis', 4: 'admixture mapping analysis'}
Rule 2: genome-wide admixture mapping analysis



Due to the nature of the sequencing method , these sequences are distributed randomly across the entire genome .
{4: 'sequencing method'}
Rule 4: sequencing method



In this method , the data are assumed to be derived from a Brownian motion model of evolution and the n samples are converted to n−1 differences between adjacent nodes in the tree , with the variance of the differences computed according to the branch lengths of the tree .
{2: 'Brownian motion model', 4: 'motion model'}
Rule 2: Brownian motion model



In short , the ANOVA model decomposes the probe signals for a given probe set into log = m+Pj+Ai+PAij+ei+eij , where yij is the hybridization signal of the jth probe o




We have developed an algorithm that uses the coordinates of all the exact matches or high similarity local alignments , clusters them with respect to the main diagonal in the dot plot using a weighted linear regression technique , and identifies the starting and ending coordinates of the region of interest .
{2: 'weighted linear regression technique', 4: 'regression technique'}
Rule 2: weighted linear regression technique



Note that one can not use all 31 predictor time points in this classical regression approach , since then the number of predictors would be larger than the sample size which is not possible .
{2: 'classical regression approach', 4: 'regression approach'}
Rule 2: classical regression approach



The top three hits for each reference gene against each comparison genome are stored in the database including details such as a gene identifier , alignment start and end , and BLAST scores such as e-value , percent identity and bit score .
{4: 'bit score'}
Rule 4: bit sc

As in our previous investigation , logistic regression analysis was used to test for the main effects of each marker locus .
{2: 'logistic regression analysis', 4: 'regression analysis'}
Rule 2: logistic regression analysis



The LRT test statistic was calculated as twice the difference between the likelihoods of the model fitting the QTL and without fitting the QTL .
{4: 'LRT test statistic'}
Rule 4: LRT test statistic



The method is developed for the allelic association analysis of pooled DNA samples , but it can be easily generalized to the analysis of individually genotyped samples .
{2: 'allelic association analysis', 4: 'association analysis'}
Rule 2: allelic association analysis



Comparison of microarray gene level signals with real-time PCR data suggests that the Globin PNAs-Affymetrix method produces the most accurate microarray results , although , all methods produce data that correlate well with the RT-PCR measurements .
{2: 'Globin PNAs-Affymetrix method', 4: 'Globin 

Rule 4: HAPPY technique



Furthermore , universal kriging is based on a polynomial trend surface model which is to be removed prior to estimation of the semivariogram from the residuals , i.e. δ .
{2: 'polynomial trend surface model', 4: 'trend surface model'}
Rule 2: polynomial trend surface model



The P-value was obtained from a likelihood ratio test based on Monte Carlo simulation with 9,999 replicates .
{4: 'likelihood ratio test'}
Rule 4: likelihood ratio test



A simple GIS based model , based on an assumed exponential decay distribution of patients around a general practice , can be used to predict a practice population-weighted area-based deprivation measure .
{2: 'simple GIS based model', 4: 'GIS model'}
Rule 2: simple GIS based model



The zones were created via k-means clustering of the results of a geographically weighted regression analysis of pathogen distributions , as documented in a previous study .
{2: 'weighted regression analysis', 4: 'regression analysis'}
Rul




The terms contributing to the jump score are illustrated in Figure 12 .
{4: 'jump score'}
Rule 4: jump score



The leave-one-out cross-validation analysis of ethnic group affiliation was performed using the GeneClass 2 software .
{2: 'leave-one-out cross-validation analysis', 4: 'cross-validation analysis'}
Rule 2: leave-one-out cross-validation analysis



This amounted to 188,600,000 simulated case control calculations for each disease model examined .
{4: 'disease model'}
Rule 4: disease model



Briefly , the systematic hybrid LOH method utilizes a set of 16 strains , each containing a conditional centromere construct on one of the 16 yeast chromosomes .
{2: 'systematic hybrid LOH method', 4: 'hybrid LOH method'}
Rule 2: systematic hybrid LOH method



The AML trend test was not significant under either scenario .
{4: 'AML trend test'}
Rule 4: AML trend test



The whole genome approach based on fitting multiple SNPs and using RJMCMC for model selection provides a posterior den

In [6]:
def writeIOB2File(inputList, filename):
    with open(filename, 'w+') as fout:
        for i in inputList:
            for j in i:
                output = ""
                if '.' in j[0]:
                    output=str(j[0]) + '\t' + str(j[1]) + '\t' + str(j[2]) + '\n\n'
                else:
                    output=str(j[0]) + '\t' + str(j[1]) + '\t' + str(j[2]) + '\n'
                fout.write(output)
                
writeIOB2File(iob_list, "complex_rules_iob_input.txt")

In [7]:
import pickle

with open('complex_rules_iob_input.pkl', 'wb') as f:
    pickle.dump(iob_list, f) 

In [8]:
print(len(iob_list))

1338
